In [6]:
import os

os.chdir("..")
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\Vanshika Garg\healthcare-risk-provider-analytics


In [7]:
import pandas as pd

patients = pd.read_csv("data/raw/patients.csv")
encounters = pd.read_csv("data/raw/encounters.csv")
providers = pd.read_csv("data/raw/providers.csv")
readmissions = pd.read_csv("data/raw/readmissions.csv")


In [8]:
patients.shape

(1000, 5)

In [9]:
encounters.shape

(3000, 6)

In [10]:
providers.shape

(50, 3)

In [11]:
readmissions.shape

(3000, 2)

In [12]:
patients.columns

Index(['patient_id', 'age', 'gender', 'chronic_condition', 'region'], dtype='object')

In [13]:
encounters.columns

Index(['encounter_id', 'patient_id', 'provider_id', 'admit_date',
       'length_of_stay', 'discharge_date'],
      dtype='object')

In [14]:
providers.columns

Index(['provider_id', 'department', 'experience_years'], dtype='object')

In [15]:
readmissions.columns

Index(['encounter_id', 'readmitted_30_days'], dtype='object')

In [16]:
patients["patient_id"].is_unique        # should be True

True

In [17]:

encounters["encounter_id"].is_unique    # should be True

True

In [18]:
df = (
    encounters
    .merge(patients, on="patient_id", how="left")
    .merge(readmissions, on="encounter_id", how="left")
    .merge(providers, on="provider_id", how="left")
)


In [19]:
df.shape

(3000, 13)

In [20]:
df.columns

Index(['encounter_id', 'patient_id', 'provider_id', 'admit_date',
       'length_of_stay', 'discharge_date', 'age', 'gender',
       'chronic_condition', 'region', 'readmitted_30_days', 'department',
       'experience_years'],
      dtype='object')

In [21]:
df.head(10)

,encounter_id,patient_id,provider_id,admit_date,length_of_stay,discharge_date,age,gender,chronic_condition,region,readmitted_30_days,department,experience_years
0,1,213,16,2023-08-17,7,2023-08-24,86,Male,Hypertension,South,1,General Medicine,29
1,2,310,28,2023-08-28,8,2023-09-05,74,Male,Hypertension,North,0,Cardiology,15
2,3,661,48,2023-12-29,1,2023-12-30,84,Male,Diabetes,North,1,Orthopedics,16
3,4,968,35,2023-11-06,3,2023-11-09,18,Female,NaN,South,0,General Medicine,5
4,5,753,45,2023-05-21,7,2023-05-28,41,Male,NaN,North,0,Cardiology,22
5,6,272,46,2023-06-05,13,2023-06-18,63,Male,Heart Disease,East,0,Cardiology,4
6,7,571,8,2023-10-26,9,2023-11-04,23,Female,Hypertension,South,1,Neurology,5
7,8,799,44,2023-12-11,8,2023-12-19,41,Male,NaN,East,0,Neurology,15
8,9,938,22,2023-04-09,8,2023-04-17,42,Female,Diabetes,South,0,Neurology,4
9,10,202,21,2023-02-16,9,2023-02-25,67,Male,Diabetes,East,1,Cardiology,15


In [22]:
df["encounter_id"].is_unique

True

In [23]:
df.isnull().sum()

encounter_id            0
patient_id              0
provider_id             0
admit_date              0
length_of_stay          0
discharge_date          0
age                     0
gender                  0
chronic_condition     878
region                  0
readmitted_30_days      0
department              0
experience_years        0
dtype: int64

In [24]:
# FEATURE ENGINEERING
#marking age group

df["is_elderly"]= df["age"]>=60

In [25]:
df["has_chronic_cond"]= df["chronic_condition"] !="None"

In [26]:
df.head()

,encounter_id,patient_id,provider_id,admit_date,length_of_stay,discharge_date,age,gender,chronic_condition,region,readmitted_30_days,department,experience_years,is_elderly,has_chronic_cond
0,1,213,16,2023-08-17,7,2023-08-24,86,Male,Hypertension,South,1,General Medicine,29,True,True
1,2,310,28,2023-08-28,8,2023-09-05,74,Male,Hypertension,North,0,Cardiology,15,True,True
2,3,661,48,2023-12-29,1,2023-12-30,84,Male,Diabetes,North,1,Orthopedics,16,True,True
3,4,968,35,2023-11-06,3,2023-11-09,18,Female,NaN,South,0,General Medicine,5,False,True
4,5,753,45,2023-05-21,7,2023-05-28,41,Male,NaN,North,0,Cardiology,22,False,True


In [27]:
visit_counts = df.groupby("patient_id")["encounter_id"].transform("count")
df["frequent_visitor"]= visit_counts>=3

In [28]:
df[["is_elderly", "has_chronic_cond", "frequent_visitor"]].mean()*100


is_elderly           42.7
has_chronic_cond    100.0
frequent_visitor     78.7
dtype: float64

In [29]:
df["chronic_condition"].value_counts()


chronic_condition
Diabetes         800
Hypertension     703
Heart Disease    619
Name: count, dtype: int64

In [30]:
df["has_chronic_condition"] = df["chronic_condition"].str.strip().str.lower() != "none"


In [31]:
df[["is_elderly", "has_chronic_condition", "frequent_visitor"]].mean()


is_elderly               0.427
has_chronic_condition    1.000
frequent_visitor         0.787
dtype: float64

In [32]:
df["chronic_condition"].value_counts(dropna=False)


chronic_condition
NaN              878
Diabetes         800
Hypertension     703
Heart Disease    619
Name: count, dtype: int64

In [33]:
df["has_chronic_condition"] = df["chronic_condition"].notna()


In [34]:
df[["is_elderly", "has_chronic_condition", "frequent_visitor"]].mean()*100


is_elderly               42.700000
has_chronic_condition    70.733333
frequent_visitor         78.700000
dtype: float64

In [35]:
#Compare readmission rates
rates = df.groupby("is_elderly")["readmitted_30_days"].mean()

elderly_rate = rates.loc[True]
non_elderly_rate = rates.loc[False]

risk_ratio = elderly_rate / non_elderly_rate

print(f"Elderly readmission rate: {elderly_rate:.2%}")
print(f"Non-elderly readmission rate: {non_elderly_rate:.2%}")
print(f"Elderly patients show ~{risk_ratio:.2f}× higher readmission probability.")


Elderly readmission rate: 52.69%
Non-elderly readmission rate: 31.82%
Elderly patients show ~1.66× higher readmission probability.


In [36]:
rates = df.groupby("frequent_visitor")["readmitted_30_days"].mean()

freq_rate = rates.loc[True]
non_freq_rate = rates.loc[False]

risk_ratio = freq_rate / non_freq_rate

print(f"Frequent visitor readmission rate: {freq_rate:.2%}")
print(f"Non-frequent visitor readmission rate: {non_freq_rate:.2%}")
print(f"Frequent visitors show ~{risk_ratio:.2f}× higher readmission risk.")


Frequent visitor readmission rate: 41.13%
Non-frequent visitor readmission rate: 39.28%
Frequent visitors show ~1.05× higher readmission risk.


In [37]:
#RISK SCORING
#ASSUMPTION
df["risk_score"] = (
    df["is_elderly"].astype(int) * 0.3 +
    df["has_chronic_condition"].astype(int) * 0.4 +
    df["frequent_visitor"].astype(int) * 0.3
)


In [38]:
df["risk_category"] = pd.cut(
    df["risk_score"],
    bins=[-1, 0.3, 0.6, 1],
    labels=["Low", "Medium", "High"]
)


In [41]:
df.groupby("risk_category", observed=True)["readmitted_30_days"].mean()*100



risk_category
Low       11.591356
Medium    36.322188
High      50.409165
Name: readmitted_30_days, dtype: float64

In [42]:
treatments=pd.read_csv("data/raw/treatments.csv")

In [43]:
#COSTING
cost_per_encounter=(
    treatments.groupby("encounter_id")["treatment_cost"]
    .sum()
    .reset_index()
)

In [44]:
df = df.merge(cost_per_encounter, on="encounter_id", how="left")


In [46]:
df.groupby("risk_category", observed=True)["treatment_cost"].mean()


risk_category
Low        97746.601179
Medium    102459.370821
High      103501.082379
Name: treatment_cost, dtype: float64

In [47]:
#PATIENT-LEVEL RISK SUMMARY

patient_risk = (
    df.groupby("patient_id")
      .agg(
          max_risk_score=("risk_score", "max"),
          risk_category=("risk_category", "max"),
          total_encounters=("encounter_id", "count"),
          avg_treatment_cost=("treatment_cost", "mean")
      )
      .reset_index()
)


In [48]:
df.head()

,encounter_id,patient_id,provider_id,admit_date,length_of_stay,discharge_date,age,gender,chronic_condition,region,readmitted_30_days,department,experience_years,is_elderly,has_chronic_cond,frequent_visitor,has_chronic_condition,risk_score,risk_category,treatment_cost
0,1,213,16,2023-08-17,7,2023-08-24,86,Male,Hypertension,South,1,General Medicine,29,True,True,False,True,0.7,High,59649
1,2,310,28,2023-08-28,8,2023-09-05,74,Male,Hypertension,North,0,Cardiology,15,True,True,True,True,1.0,High,107986
2,3,661,48,2023-12-29,1,2023-12-30,84,Male,Diabetes,North,1,Orthopedics,16,True,True,True,True,1.0,High,88749
3,4,968,35,2023-11-06,3,2023-11-09,18,Female,NaN,South,0,General Medicine,5,False,True,True,False,0.3,Low,146295
4,5,753,45,2023-05-21,7,2023-05-28,41,Male,NaN,North,0,Cardiology,22,False,True,True,False,0.3,Low,184238


In [49]:
patient_risk.to_csv(
    "data/processed/patient_risk_summary.csv",
    index=False
)